This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [25]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv('NASDAQ_API_KEY')

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
from datetime import datetime
import os
from dotenv import load_dotenv

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
# Construct the API URL
base_url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X"
params = {
    "start_date": "2017-01-02",
    "end_date": "2017-01-02",
    "api_key": API_KEY
}

# Make the API request
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a Python dictionary
    data = response.json()
    
    # Print the JSON structure
    print(json.dumps(data, indent=2))
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")

Error: Unable to fetch data. Status code: 410


In [6]:
import requests
import json
import os

# Get the API key from an environment variable
API_KEY = os.getenv('NASDAQ_API_KEY')

# Construct the API URL
base_url = "https://data.nasdaq.com/api/v3/datasets/FRED/GDP.json"
params = {
    "start_date": "2022-01-01",
    "end_date": "2022-12-31",
    "api_key": API_KEY
}

# Make the API request
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a Python dictionary
    data = response.json()
    
    # Print the JSON structure
    print(json.dumps(data, indent=2))
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")
    print(f"Response content: {response.text}")

Error: Unable to fetch data. Status code: 410
Response content: Resource is no longer available. Path: /api/v3/datasets/FRED/GDP.json


In [24]:
import os
from dotenv import load_dotenv
import json
from datetime import datetime, timedelta
API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')

load_dotenv()

True

In [14]:
import requests
base_url = "https://www.alphavantage.co/query"
params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": "AFX.DEX",  # AFX stock on the Deutsche Börse (similar to FSE)
    "outputsize": "full",
    "apikey": API_KEY
}

# Make the API request
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a Python dictionary
    data = response.json()
    
    # Extract data for 2017-01-02 (or the nearest available trading day)
    time_series = data.get('Time Series (Daily)', {})
    target_date = datetime(2017, 1, 2)
    
    while target_date.strftime('%Y-%m-%d') not in time_series:
        target_date += timedelta(days=1)
        if target_date.year > 2017:
            print("No data found for the specified date range.")
            exit()

    sample_data = {
        target_date.strftime('%Y-%m-%d'): time_series[target_date.strftime('%Y-%m-%d')]
    }
    
    # Create a structure similar to the original Nasdaq/Quandl API
    formatted_data = {
        "dataset": {
            "dataset_code": "AFX_X",
            "database_code": "FSE",
            "name": "AFX Stock Data",
            "data": [
                [
                    target_date.strftime('%Y-%m-%d'),
                    float(sample_data[target_date.strftime('%Y-%m-%d')]['1. open']),
                    float(sample_data[target_date.strftime('%Y-%m-%d')]['2. high']),
                    float(sample_data[target_date.strftime('%Y-%m-%d')]['3. low']),
                    float(sample_data[target_date.strftime('%Y-%m-%d')]['4. close']),
                    int(float(sample_data[target_date.strftime('%Y-%m-%d')]['5. volume']))
                ]
            ],
            "column_names": [
                "Date",
                "Open",
                "High",
                "Low",
                "Close",
                "Volume"
            ]
        }
    }
    
    # Print the JSON structure
    print(json.dumps(formatted_data, indent=2))
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")
    print(f"Response content: {response.text}")

{
  "dataset": {
    "dataset_code": "AFX_X",
    "database_code": "FSE",
    "name": "AFX Stock Data",
    "data": [
      [
        "2017-01-02",
        34.99,
        35.94,
        34.99,
        35.8,
        44700
      ]
    ],
    "column_names": [
      "Date",
      "Open",
      "High",
      "Low",
      "Close",
      "Volume"
    ]
  }
}


In [17]:
import json
from pprint import pprint

# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_data = {
  "dataset": {
    "dataset_code": "AFX_X",
    "database_code": "FSE",
    "name": "Carl Zeiss Meditec (AFX_X)",
    "description": "Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704",
    "refreshed_at": "2020-12-01T14:48:09.907Z",
    "newest_available_date": "2020-12-01",
    "oldest_available_date": "2000-06-07",
    "column_names": [
      "Date",
      "Open",
      "High",
      "Low",
      "Close",
      "Change",
      "Traded Volume",
      "Turnover",
      "Last Price of the Day",
      "Daily Traded Units",
      "Daily Turnover"
    ],
    "frequency": "daily",
    "type": "Time Series",
    "premium": False,
    "limit": None,
    "transform": None,
    "column_index": None,
    "start_date": "2021-01-03",
    "end_date": "2020-12-01",
    "data": [],
    "collapse": None,
    "order": None,
    "database_id": 6129
  }
}

# Add the missing 'id' field
json_data['dataset']['id'] = 10095370

# Pretty print the JSON structure
pprint(json_data)

{'dataset': {'collapse': None,
             'column_index': None,
             'column_names': ['Date',
                              'Open',
                              'High',
                              'Low',
                              'Close',
                              'Change',
                              'Traded Volume',
                              'Turnover',
                              'Last Price of the Day',
                              'Daily Traded Units',
                              'Daily Turnover'],
             'data': [],
             'database_code': 'FSE',
             'database_id': 6129,
             'dataset_code': 'AFX_X',
             'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) '
                            'from the Frankfurt Stock Exchange.<br><br>Trading '
                            'System: Xetra<br><br>ISIN: DE0005313704',
             'end_date': '2020-12-01',
             'frequency': 'daily',
             'id': 

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [21]:
base_url = "https://www.alphavantage.co/query"
params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": "AFX.DEX",  # AFX stock on the Deutsche Börse (similar to FSE)
    "outputsize": "full",
    "apikey": "API_KEY"  
}

# 1. Collect data 
response = requests.get(base_url, params=params)

# 2. Convert the returned JSON object into a Python dictionary
if response.status_code == 200:
    data = response.json()
    
    # Extract the time series data
    time_series = data.get('Time Series (Daily)', {})
    
    # 3. Calculate the highest and lowest opening prices for 2017
    opening_prices_2017 = [
        float(daily_data['1. open'])
        for date, daily_data in time_series.items()
        if date.startswith('2017')
    ]
    
    if opening_prices_2017:
        highest_opening = max(opening_prices_2017)
        lowest_opening = min(opening_prices_2017)
        
        print(f"Highest opening price in 2017: {highest_opening}")
        print(f"Lowest opening price in 2017: {lowest_opening}")
    else:
        print("No opening price data available for 2017.")
    
    # Print the first few rows of 2017 data to verify
    print("\nFirst few rows of 2017 data:")
    count = 0
    for date, daily_data in time_series.items():
        if date.startswith('2017'):
            print(f"{date}: Open: {daily_data['1. open']}")
            count += 1
            if count == 5:
                break
else:
    print(f"Error fetching data: {response.status_code}")
    print(response.text)

Highest opening price in 2017: 53.11
Lowest opening price in 2017: 34.0

First few rows of 2017 data:
2017-12-29: Open: 51.7600
2017-12-28: Open: 51.6500
2017-12-27: Open: 51.4500
2017-12-22: Open: 51.0500
2017-12-21: Open: 51.1600


In [23]:
import statistics
if response.status_code == 200:
    data = response.json()
    
    # Extract the time series data
    time_series = data.get('Time Series (Daily)', {})
    
    # Filter data for 2017
    data_2017 = {date: daily_data for date, daily_data in time_series.items() if date.startswith('2017')}
    
    # 3. Calculate the highest and lowest opening prices for 2017
    opening_prices = [float(daily_data['1. open']) for daily_data in data_2017.values()]
    highest_opening = max(opening_prices)
    lowest_opening = min(opening_prices)
    
    print(f"Highest opening price in 2017: {highest_opening}")
    print(f"Lowest opening price in 2017: {lowest_opening}")
    
    # 4. Largest change in any one day
    daily_changes = [float(daily_data['2. high']) - float(daily_data['3. low']) for daily_data in data_2017.values()]
    largest_daily_change = max(daily_changes)
    print(f"Largest change in any one day: {largest_daily_change}")
    
    # 5. Largest change between any two days
    closing_prices = [float(daily_data['4. close']) for daily_data in data_2017.values()]
    closing_changes = [abs(closing_prices[i] - closing_prices[i+1]) for i in range(len(closing_prices)-1)]
    largest_two_day_change = max(closing_changes)
    print(f"Largest change between any two days: {largest_two_day_change}")
    
    # 6. Average daily trading volume
    volumes = [int(daily_data['5. volume']) for daily_data in data_2017.values()]
    avg_volume = sum(volumes) / len(volumes)
    print(f"Average daily trading volume: {avg_volume:.2f}")
    
    # 7. Median trading volume
    median_volume = statistics.median(volumes)
    print(f"Median trading volume: {median_volume}")

else:
    print(f"Error fetching data: {response.status_code}")
    print(response.text)

Highest opening price in 2017: 53.11
Lowest opening price in 2017: 34.0
Largest change in any one day: 2.8100000000000023
Largest change between any two days: 2.5549999999999997
Average daily trading volume: 92684.38
Median trading volume: 78178.0
